
# **1. Question or problem definition**

อยากจะรู้ถ้า overall ราคา ค่าเหนื่อย เท่านี้ แล้วนักเตะควรจะมี potential เท่าไหร่ และคุ้มไหม

![](https://versus.uk.com/wp-content/uploads/2020/01/versus-future-stars.jpg)> 

# **2. Acquire training and testing data.**

In [ ]:

import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn import linear_model# compute classification accuracy for the linear regression model
from sklearn import metrics # for the check the error and accuracy of the model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
df20 = pd.read_csv('/kaggle/input/fifa-20-complete-player-dataset/players_20.csv')


# **3.Wrangle, prepare, cleanse the data.**

ทำสำรองข้อมูลไว้

In [ ]:

originalfifa = df20.copy()
df20.head()

โดยผมจะบอกก่อนให้เข้าใจว่าค่าที่นักเตะสามารถพัฒนาไปได้สูงสุดคือ potential และค่าพลังรวมนักเตะตอนนี้คือ overall

เริ่มเช็คความเสียหายของ database หรือช่องว่าง

In [ ]:

df20.info()

In [ ]:
df20.isnull().any()

ทำการเติมค่าลงไปที่ช่องที่ขาดหาย หรือข้อมูลที่ขาดหาย

In [ ]:
df20 = df20.fillna(value=0, axis=1)
df20.isnull().any()

In [ ]:
df20.shape

ดูปีที่ตั้งเตะเข้าร่วมทีมที่อยู่ เพื่อแยกว่านักเตะคนไหนที่ซื้อไม่ได้บ้าง

In [ ]:
df20['joined'].head()

แยกให้เหลือแต่ปี

In [ ]:

dt_series = pd.to_datetime(df20['joined'])
df20['year'] = dt_series.dt.year
df20.head()

แยกว่านักเตะคนไหนที่ซื้อไม่ได้บ้าง

In [ ]:
def joined_condition(year):
    if year == 2019.0:
        return 0
    else:
        return 1

df20['joined_status'] = df20['year'].apply(joined_condition)
df20.head()

ต้องการแยกนักเตะที่มีอายุมากกว่า 25 ปีขึ้นไปเพราะไม่เหมาะกับการนำมาพัฒนา ทำฟังก์ชั่นและคอลัม
เนื่องจากข้อมูลเยอะไปจึงต้องตัดข้อมูลออก

In [ ]:
sns.countplot(x="age", data=df20);

จะเห็นได้ว่ามีข้อมูลที่มีอายุมากกว่า 25 มีเยอะมากและไม่จำเป็นเราจึงจะตัดออกไปและ ตัดนักเตะที่ไม่พร้อมย้ายออกไป

In [ ]:
sns.countplot(x="year", data=df20);

จะเห็นได้ว่ามีนักเตะปี 2019 คือไม่พร้อมย้ายเยอะมาก

In [ ]:

df20 = df20[df20['age']< 25]
df20['age'].value_counts()
df20 = df20[df20['joined_status']==1]
df20['joined_status'].value_counts()
df20.head()

เริ่มตัดนักเตะที่ไม่พร้อมย้ายทีมให้เหลือแต่ที่พร้อม

In [ ]:
sns.countplot(x="year", data=df20);

In [ ]:
sns.countplot(x="age", data=df20);

In [ ]:
df20 = df20[df20['year']!= 2019]
df20['year'].value_counts()

เช็คว่าข้อมูลมีความเสียหายหรือไม่

In [ ]:
df20.replace([np.inf, -np.inf], np.nan)
df20.isnull().any()

จะเห็นได้ว่าไม่มีข้อมูลเสียหายแล้ว

# **4.Analyze, identify patterns, and explore the data.******

คัดเฉพราะหัวข้อที่ต้องใช้ไว้ ส่วนที่ไม่จำเป็นตัดทิ้ง

In [ ]:
# คัดเลือก feature ด้วยความรู้สึกของตัวเอง
feature_selected = ['short_name', 'long_name','age','overall', 'potential', 'value_eur', 'wage_eur', 'player_positions','joined_status']
df20 = df20[feature_selected]
df20.head()

ตรวจสอบความสัมพันธ์

In [ ]:
sns.heatmap(df20.corr(), annot=True)

จะเห็นว่า heatmap ออกมาไม่แย่

แสดงชนิดของข้อมูล

In [ ]:
df20.dtypes

In [ ]:
sns.lmplot(x='overall',y='potential',data=df20,ci=None,scatter_kws={'alpha':0.4},line_kws={'color':'orange'})

In [ ]:
จะเห็นได้ว่า overall และ potential ไปในทิศทางเดียวกัน

In [ ]:
sns.lmplot(x='value_eur',y='potential',data=df20,ci=None,scatter_kws={'alpha':0.4},line_kws={'color':'orange'})

จะเห็นได้ว่า value_eur และ potential ไปในทิศทางเดียวกันในช่วงกลางๆ

In [ ]:
sns.lmplot(x='wage_eur',y='potential',data=df20,ci=None,scatter_kws={'alpha':0.4},line_kws={'color':'orange'})

ทำการจัดข้อมูลเพื่อทำขั้นตอนที่ 5

In [ ]:
categories = ['short_name', 'long_name','player_positions']
raw_model_data = pd.get_dummies(df20.copy(), columns=categories,drop_first=True)

# **5.Model, predict and solve the problem.**

ใช้ linear regression ในการทำนายข้อมูล

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import linear_model
import json

โดย target จะเท่ากับ potential และ features เท่ากับ overall,value_eur,wage_eur,age

In [ ]:
features = ['overall','value_eur','wage_eur','age']
target = ['potential']

In [ ]:
train, test = train_test_split(df20,test_size=0.2,random_state=5)
train.head()

นำข้อมูลที่ต้องการใส่ลงใน train และ test

In [ ]:
X_train = train[features].dropna()
y_train = train[target].dropna()
X_test = test[features].dropna()
y_test = test[target].dropna()

ทำการแสดงขนาดส่วนของ train และ test

In [ ]:
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

In [ ]:

lin = linear_model.LinearRegression()
lin.fit(X_train, y_train)

In [ ]:
lin_score_train = lin.score(X_test, y_test)
lin_score_test = lin.score(X_train, y_train)
print("Training score: ",lin_score_train)
print("Testing score: ",lin_score_test)

จะเห็นได้ว่า linear Regression มีความแม่นยำถึง 87 % เราจึงใช้ตัวนี้ในการ predict ต่อไป

In [ ]:
y_hat = lin.predict(X_test)
y_hat

# ** 6.Visualize, report, and present the problem solving steps and final solution.******

In [ ]:

promising_talents2 = test.loc[(test['potential']>85) & (test['age']<25)]
fig = go.Figure()
fig.add_trace(go.Scatter(x=promising_talents2.value_eur, y=promising_talents2.potential,
                    mode='lines+markers',
                    ))
fig.update_layout(title=' Best Worthiness player values(Euros)',
                   xaxis_title='player name ',
                   yaxis_title='Potential')
fig.show()


In [ ]:
test.head()

In [ ]:
print("X_value = %s, Predicted=%s" % (X_test, y_hat[0]))


เมื่อลอง test ดูแล้ว ค่าที่ predict มามีความใกล้เคียงกับข้อมูลแรกมาก 

![](https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.fifaultimateteam.it%2Fen%2Ffifa20-rating-tile-full-rating-vinicius-jr%2F&psig=AOvVaw10eDSTNczM3VLLiBGwJBtT&ust=1586624337729000&source=images&cd=vfe&ved=0CAIQjRxqFwoTCPCQlraq3ugCFQAAAAAdAAAAABAD)

![](https://www.fifaultimateteam.it/en/wp-content/uploads/2019/09/fifa20-rating-tile-full-rating-vinicius-jr.png)

![](https://outsidergaming.com/wp-content/uploads/2020/01/Lautaro-Martinez-Inter-Milan-FIFA-20-Best-Argentinean-Wonderkids.jpg)